In [1]:
!pip install ultralytics


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: c:\Users\Shreyas\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
import cv2
import shutil
import os
import json
from datetime import datetime
from ultralytics import YOLO


model_path = r"D:\TCS\Road_Damage_Detection\Yolov8m\runforyolov8m\content\runforyolov8m\training_yolov8m\weights\best.pt"
input_folder = r"D:\TCS\Road_Damage_Detection\test\images"
output_folder = r"D:/TCS/Road_Damage_Detection/test/output"
archive_folder = r"D:/TCS/Road_Damage_Detection/test/archive"
json_folder = r"D:/TCS/Road_Damage_Detection/test/jsonfiles"
CONF_THRESHOLD = 0.25


os.makedirs(output_folder, exist_ok=True)
os.makedirs(archive_folder, exist_ok=True)
os.makedirs(json_folder, exist_ok=True)

print(f"Loading model from: {model_path}")
model = YOLO(model_path)

print(f"Scanning input folder: {input_folder}")

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.webp')):
        image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, filename)
        archive_image_path = os.path.join(archive_folder, filename)
        json_file_path = os.path.join(json_folder, os.path.splitext(filename)[0] + ".json")

        print(f"\nProcessing image: {filename}")

        image = cv2.imread(image_path)
        if image is None:
            print(f"ERROR: Could not read {image_path}")
            continue

        
        results = model(image_path)
        result = results[0]

        detections = []

        # Process detections
        if result.boxes is not None and len(result.boxes) > 0:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                conf = float(box.conf[0])
                cls = int(box.cls[0])

                if conf >= CONF_THRESHOLD:
                    class_name = model.names[cls]
                    # Draw bounding boxes
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(image, f"{class_name} {conf:.2f}", (x1, max(y1 - 10, 20)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                    # Append to detections list
                    detections.append({
                        "x": x1,
                        "y": y1,
                        "width": x2 - x1,
                        "height": y2 - y1,
                        "confidence": round(conf, 3),
                        "class": class_name,
                        "class_id": cls
                    })

        # Save annotated image if detections exist
        if len(detections) > 0:
            cv2.imwrite(output_image_path, image)
            print(f"✅ Damage detected! Saved annotated image: {output_image_path}")
        else:
            print(f"No damage detected in {filename}")

        # Save JSON file with detections and timestamp
        detection_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        json_data = {
            "image_name": filename,
            "model_used": os.path.basename(model_path),
            "timestamp": detection_time,
            "detections": detections if detections else "No damage detected"
        }

        with open(json_file_path, "w", encoding="utf-8") as jf:
            json.dump(json_data, jf, indent=4)
        print(f"📝 JSON saved: {json_file_path}")

        # Move processed image to archive
        try:
            shutil.move(image_path, archive_image_path)
            print(f"Moved processed file to: {archive_image_path}")
        except Exception as e:
            print(f"Could not move {filename}: {e}")

print("\n✅ Processing complete!")


Loading model from: D:\TCS\Road_Damage_Detection\Yolov8m\runforyolov8m\content\runforyolov8m\training_yolov8m\weights\best.pt
Scanning input folder: D:\TCS\Road_Damage_Detection\test\images

Processing image: 12_jpg.rf.581e3d2ede69811df50b557ab6379824.jpg

image 1/1 D:\TCS\Road_Damage_Detection\test\images\12_jpg.rf.581e3d2ede69811df50b557ab6379824.jpg: 640x640 3 alligator crackings, 3 longitudinal crackings, 564.5ms
Speed: 12.0ms preprocess, 564.5ms inference, 8.4ms postprocess per image at shape (1, 3, 640, 640)
✅ Damage detected! Saved annotated image: D:/TCS/Road_Damage_Detection/test/output\12_jpg.rf.581e3d2ede69811df50b557ab6379824.jpg
📝 JSON saved: D:/TCS/Road_Damage_Detection/test/jsonfiles\12_jpg.rf.581e3d2ede69811df50b557ab6379824.json
Moved processed file to: D:/TCS/Road_Damage_Detection/test/archive\12_jpg.rf.581e3d2ede69811df50b557ab6379824.jpg

Processing image: 17_jpg.rf.4d81ebeb51289dc3c80ac75c10a5ac7f.jpg

image 1/1 D:\TCS\Road_Damage_Detection\test\images\17_jpg.rf.4d